In [1]:
import ouster
from ouster import pcap
from ouster import client
import matplotlib.pyplot as plt
from more_itertools import nth 
import open3d as o3d
import numpy as np
import os
import cv2

from mmdet3d.apis import init_model, inference_detector, show_result_meshlab

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


ModuleNotFoundError: No module named 'mmdet3d'

In [ ]:
# start wih sample data
pcap_path = 'dataset\OS0-128\OS0-128_Rev-06_fw23_Urban-Drive_Dual-Returns.pcap'
metadata_path = 'dataset\OS0-128\OS0-128_Rev-06_fw23_Urban-Drive_Dual-Returns.json'

with open(metadata_path, 'r') as f:
    info = client.SensorInfo(f.read())
source = pcap.Pcap(pcap_path, info)

scans = client.Scans(source)
scan = nth(scans, 10)

In [3]:
def vis_point_cloud(scan):
    if isinstance(scan, ouster.client._client.LidarScan):
        xyz = client.XYZLut(info)(scan)
        # create point cloud and coordinate axes geometries
        cloud = o3d.geometry.PointCloud(
            o3d.utility.Vector3dVector(xyz.reshape((-1, 3))))  # type: ignore
    else:
        cloud = scan
    axes = o3d.geometry.TriangleMesh.create_coordinate_frame(
        1.0)  # type: ignore
     # initialize visualizer and rendering options
    vis = o3d.visualization.Visualizer()  # type: ignore

    vis.create_window()
    vis.add_geometry(cloud)
    vis.add_geometry(axes)
    ropt = vis.get_render_option()
    ropt.point_size = 1.0
    ropt.background_color = np.asarray([0, 0, 0])

    # initialize camera settings
    ctr = vis.get_view_control()
    ctr.set_zoom(0.1)
    ctr.set_lookat([0, 0, 0])
    ctr.set_up([1, 0, 0])

    # run visualizer main loop
    print("Press Q or Excape to exit")
    vis.run()
    vis.destroy_window()

In [5]:
xyz = client.XYZLut(info)(scan)
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(xyz.reshape((-1, 3))))

In [6]:
# Define a threshold:
THRESHOLD = 1

# Get the min value along the z-axis:
z_min = min(pcd.points, key=lambda x: x[2])[2]

select_idx = []

for i, p in enumerate(pcd.points):
    p_x = p[0]
    p_y = p[1]
    p_z = p[2]
    
    if abs(p_x) < 5 and abs(p_y) < 2 and abs(p_z) < 2:
        select_idx.append(i)
    
    # if p_z - z_min < THRESHOLD:
    #     select_idx.append(i)

select_roi = pcd.select_by_index(select_idx)

In [8]:
vis_point_cloud(select_roi)

Press Q or Excape to exit


In [ ]:
# config_file = './configs/second/hv_second_secfpn_6x8_80e_kitti-3d-3class.py'
# checkpoint_file = 'D:/Code/Jiuzhou/mmdetection3d/models/second/hv_second_secfpn_6x8_80e_kitti-3d-3class.pth'

In [ ]:
# model = init_model(config_file, checkpoint_file, device='cuda:0')

In [ ]:
# demo_pcd = 'D:/Code/Jiuzhou/mmdetection3d/demo/data/kitti/kitti_000008.bin'
# result, data = inference_detector(model, 'pcd_out_000001.pcd')

In [ ]:
# out_dir = './'
# show_result_meshlab(data, result, out_dir, show=True)

In [ ]:
# !python -m ouster.sdk.examples.pcap {pcap_path} {metadata_path} pcap-to-pcd --scan-num 5